In [8]:
pip install pydub

In [7]:
#Step 1: Mount Google Drive from google.colab import drive
drive.mount('/content/drive')

NameError: name 'drive' is not defined

In [15]:
import warnings
warnings.filterwarnings('ignore')

In [17]:
# Step 2: Load Large MP3 File (chunked processing)
import torchaudio
import librosa
import os
from pydub import AudioSegment

# Parameters
audio_path = "/content/audio_01.mp3"
chunk_length_ms = 5 * 60 * 100  # 5 minutes

# Split audio into chunks
def split_audio(path, chunk_len=chunk_length_ms):
    audio = AudioSegment.from_mp3(path)
    chunks = [audio[i:i+chunk_len] for i in range(0, len(audio), chunk_len)]
    paths = []
    os.makedirs("/content/chunks", exist_ok=True)
    for i, chunk in enumerate(chunks):
        path = f"/content/chunks/chunk_{i}.wav"
        chunk.export(path, format="wav")
        paths.append(path)
    return paths

chunk_paths = split_audio(audio_path)
print("Total chunks:", len(chunk_paths))

Total chunks: 53


In [18]:
# Step 3: Load distil-whisper/distil-large-v3.5 Model from Hugging Face
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq, pipeline
import torch

model_id = "distil-whisper/distil-large-v3.5"

device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForSpeechSeq2Seq.from_pretrained(model_id, torch_dtype=torch_dtype, use_safetensors=True).to(device)

pipe = pipeline("automatic-speech-recognition", model=model, tokenizer=processor.tokenizer,
                feature_extractor=processor.feature_extractor, device=0 if torch.cuda.is_available() else -1)


Device set to use cuda:0


In [19]:
# Step 4: Transcribe Each Chunk
transcripts = []
for i, chunk in enumerate(chunk_paths):
    print(f"Processing chunk {i+1}/{len(chunk_paths)}")
    text = pipe(chunk)["text"]
    transcripts.append(text)

Processing chunk 1/53
Processing chunk 2/53
Processing chunk 3/53
Processing chunk 4/53
Processing chunk 5/53
Processing chunk 6/53
Processing chunk 7/53
Processing chunk 8/53
Processing chunk 9/53
Processing chunk 10/53
Processing chunk 11/53
Processing chunk 12/53
Processing chunk 13/53
Processing chunk 14/53
Processing chunk 15/53
Processing chunk 16/53
Processing chunk 17/53
Processing chunk 18/53
Processing chunk 19/53
Processing chunk 20/53
Processing chunk 21/53
Processing chunk 22/53
Processing chunk 23/53
Processing chunk 24/53
Processing chunk 25/53
Processing chunk 26/53
Processing chunk 27/53
Processing chunk 28/53
Processing chunk 29/53
Processing chunk 30/53
Processing chunk 31/53
Processing chunk 32/53
Processing chunk 33/53
Processing chunk 34/53
Processing chunk 35/53
Processing chunk 36/53
Processing chunk 37/53
Processing chunk 38/53
Processing chunk 39/53
Processing chunk 40/53
Processing chunk 41/53
Processing chunk 42/53
Processing chunk 43/53
Processing chunk 44/

In [20]:
transcripts

["Hello. Hi, this is Jasper from O'Vecilan. This is the callback that you requested. Can I grab your first name and last name? First name is Ever, H-E-V-E-R. Last name is Huerta, H-U-E-R-T-A. Okay, all right. How can I help you? I got a call from a customer this morning. They had an alarm go off in ACC using the focus of attention. They noticed they come up when they went to a",
 "acknowledge it doesn't do anything it just they click it but not it doesn't actually turn off so what would cause it to do that or how do I turn it off just a moment yep basically you saw alarm on ACC but when like",
 "What was that alarm saying? The alarm is one of the doors. It says fire exit is open. It's one of the doors that we have programs through it. Okay. And when they pressed it, it doesn't do anything. Correct. And they have, they have, they've looked at the door itself to make sure it is closed and it is.",
 "Sorry, they looked at the door and it is closed. Correct. So you are saying it was kind o

In [23]:
full_transcript = "\n".join(transcripts)
with open("/content/full_transcript.txt", "w") as f:
    f.write(full_transcript)

print("Transcription complete.")

Transcription complete.


In [24]:
print(full_transcript)

Hello. Hi, this is Jasper from O'Vecilan. This is the callback that you requested. Can I grab your first name and last name? First name is Ever, H-E-V-E-R. Last name is Huerta, H-U-E-R-T-A. Okay, all right. How can I help you? I got a call from a customer this morning. They had an alarm go off in ACC using the focus of attention. They noticed they come up when they went to a
acknowledge it doesn't do anything it just they click it but not it doesn't actually turn off so what would cause it to do that or how do I turn it off just a moment yep basically you saw alarm on ACC but when like
What was that alarm saying? The alarm is one of the doors. It says fire exit is open. It's one of the doors that we have programs through it. Okay. And when they pressed it, it doesn't do anything. Correct. And they have, they have, they've looked at the door itself to make sure it is closed and it is.
Sorry, they looked at the door and it is closed. Correct. So you are saying it was kind of a false alar

In [25]:
# Step 5: Load Small Language Model (SLM) for Entity Extraction
from transformers import pipeline

# Using FLAN-T5 for simplicity
nlp_ner = pipeline("text2text-generation", model="google/flan-t5-base")

prompt = f"Extract all order-related entities from this conversation:\n\n{full_transcript}"
result = nlp_ner(prompt, max_new_tokens=512)[0]['generated_text']

print("Extracted Entities:\n", result)


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cuda:0
Token indices sequence length is longer than the specified maximum sequence length for this model (1008 > 512). Running this sequence through the model will result in indexing errors


Extracted Entities:
 O'Vecilan, customer, alarm, ACC, ACK, ACK, ATT, remote session, remote session


In [26]:
import json

structured_entities = {
    "Customer Name": "",
    "Order ID": "",
    "Product": "",
    "Quantity": "",
    "Delivery Date": ""
}

# Manual parsing or regex-based enhancement can go here
structured_entities["Raw Output"] = result

In [27]:
result

"O'Vecilan, customer, alarm, ACC, ACK, ACK, ATT, remote session, remote session"